In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('view.csv')

In [3]:
df

,Source,Location ID,City,State,Country,Latitude,Longitude,Time Zone,Elevation,Local Time Zone,DHI Units,DNI Units,GHI Units,Temperature Units,Wind Speed Units
0,NSRDB,20013,Jodhpur,Rajasthan,India,26.35,73.05,5.5,0.0,5.5,w/m2,w/m2,w/m2,c,m/s
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Hour,GHI,DHI,Temperature,Wind Speed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,0,0,11,0.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2,0,0,10,0.2,NaN,NaN,NaN,NaN,NaN,Abbreviations:,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8758,8756,0,0,15,0.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8759,8757,0,0,14,0.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8760,8758,0,0,13,0.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8761,8759,0,0,12,0.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Data Cleaning

In [4]:
df.drop(df.loc[:, 'Latitude':'Wind Speed Units'].columns, axis=1, inplace=True)
#Dropping unnecessary columns

In [5]:
df.dtypes
#Checking if the columns are iun the ideal type or not

Source         object
Location ID    object
City           object
State          object
Country        object
dtype: object

In [6]:
df.rename(columns={'Source':'Hour', 'Location ID':'GHI', 'City':'DHI', 'State':'Temperature', 'Country':'Wind Speed'}, inplace=True)
#Renaming the columns to our need

In [7]:
df.drop(df.index[:3], inplace=True)
df = df.reset_index(drop=True)
#Dropping first three unnecessary rows and resetting the index

In [8]:
for i in df.columns:
    df[i]=df[i].astype('float32')
#Changing the data types to float to ease calculations in the future

In [9]:
df

,Hour,GHI,DHI,Temperature,Wind Speed
0,1.0,0.0,0.0,11.0,0.2
1,2.0,0.0,0.0,10.0,0.2
2,3.0,0.0,0.0,10.0,0.2
3,4.0,0.0,0.0,9.0,0.2
4,5.0,0.0,0.0,9.0,0.2
...,...,...,...,...,...
8755,8756.0,0.0,0.0,15.0,0.3
8756,8757.0,0.0,0.0,14.0,0.3
8757,8758.0,0.0,0.0,13.0,0.3
8758,8759.0,0.0,0.0,12.0,0.3


## Finding BHI

In [10]:
df['BHI']=df['GHI']-df['DHI']
#Finding beam radiation I_b from I_g and I_d

In [11]:
Ib=df['BHI'].to_numpy()
Id=df['DHI'].to_numpy()
Ig=df['GHI'].to_numpy()
#Creating arrays as numpy arrays are computationally faster than pandas

## Finding I_sc'

In [12]:
day=np.arange(1,366)
df['day']=np.repeat(day,24)
#Creating the days of an year and saving them to the dataframe

In [13]:
I_sc_day = 1367*(1+0.033*np.cos(np.deg2rad(360*day/365)))
#Finding the daywise solar irradiance

In [14]:
df['I_sc\'']=np.repeat(I_sc_day, 24)
#As solar irradiance is constant for a day, we repeat the value for 24 hours and save it to the dataframe

## Finding Solar Zenith Angle and incidence angle

θ = f(β, γ, δ, ω, φ)<br>
θz = f(β=0, γ, δ, ω, φ)<br>
where β=collector tilt angle, γ=surface azimuth angle, δ=declination angle, ω=hour angle, φ=latitude<br>
Latitude is given as 26.35<br>Assuming that the panels are facing south<br>Assuming an average value of tilt angle (for now)

In [15]:
φ=26+35/60
γ=0
β=15

### Declination Angle

In [16]:
δ=23.45*np.sin(np.deg2rad(360*(284+day)/365))
#Finding the declination angle using the formula

In [17]:
δ=np.repeat(δ, 24)
df['δ']=δ
#As solar declination angle is constant for a day, we repeat the value for 24 hours and save it to the dataframe

In [18]:
hr = np.arange(0,24*365)%24+1
df['daywise_hours'] = hr
#We would want daywise hours from 1-24 to be clear about our daywise radiation

### Hour Angle

In [19]:
E=[]
for i in day:
    B=(i-1)*360/365
    E.append(229.18*(0.000075+0.001868*np.cos(np.deg2rad(B))-0.032077*np.sin(np.deg2rad(B))-0.014615*np.cos(np.deg2rad(2*B))-0.04089*np.sin(np.deg2rad(2*B))))
E=np.repeat(E,24)
df['Error in min']=E
#Finding daywise error in time and saving it to the dataframe

In [20]:
LAT=[]
for i,j in enumerate(hr):
    LAT.append(j*60-37.8+E[i])
LAT=np.asarray(LAT)
df['LAT_minutes']=LAT
#Finding LAT in minutes and saving it to the dataframe

In [21]:
df['LAT']=pd.to_datetime(df.LAT_minutes, unit='m').dt.strftime('%H:%M')
#Finding LAT in time format and saving it to the dataframe

In [22]:
ω=(LAT-720)/60*15
df['ω']=ω
#Finding hour angle in time format and saving it to the dataframe

In [23]:
df

,Hour,GHI,DHI,Temperature,Wind Speed,BHI,day,I_sc',δ,daywise_hours,Error in min,LAT_minutes,LAT,ω
0,1.0,0.0,0.0,11.0,0.2,0.0,1,1412.104316,-23.011637,1,-2.904169,19.295831,00:19,-175.176042
1,2.0,0.0,0.0,10.0,0.2,0.0,1,1412.104316,-23.011637,2,-2.904169,79.295831,01:19,-160.176042
2,3.0,0.0,0.0,10.0,0.2,0.0,1,1412.104316,-23.011637,3,-2.904169,139.295831,02:19,-145.176042
3,4.0,0.0,0.0,9.0,0.2,0.0,1,1412.104316,-23.011637,4,-2.904169,199.295831,03:19,-130.176042
4,5.0,0.0,0.0,9.0,0.2,0.0,1,1412.104316,-23.011637,5,-2.904169,259.295831,04:19,-115.176042
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,8756.0,0.0,0.0,15.0,0.3,0.0,365,1412.111000,-23.085911,20,-2.453134,1159.746866,19:19,109.936716
8756,8757.0,0.0,0.0,14.0,0.3,0.0,365,1412.111000,-23.085911,21,-2.453134,1219.746866,20:19,124.936716
8757,8758.0,0.0,0.0,13.0,0.3,0.0,365,1412.111000,-23.085911,22,-2.453134,1279.746866,21:19,139.936716
8758,8759.0,0.0,0.0,12.0,0.3,0.0,365,1412.111000,-23.085911,23,-2.453134,1339.746866,22:19,154.936716


In [24]:
def value(β):
    first = np.sin(np.deg2rad(β))*np.cos(np.deg2rad(γ))*(np.cos(np.deg2rad(δ))*np.cos(np.deg2rad(ω))*np.sin(np.deg2rad(φ))-np.sin(np.deg2rad(δ))*np.cos(np.deg2rad(φ)))
    second = np.sin(np.deg2rad(β))*np.sin(np.deg2rad(γ))*np.cos(np.deg2rad(δ))*np.sin(np.deg2rad(ω))
    third = np.cos(np.deg2rad(β))*(np.cos(np.deg2rad(δ))*np.cos(np.deg2rad(ω))*np.cos(np.deg2rad(φ))+np.sin(np.deg2rad(δ))*np.sin(np.deg2rad(φ)))                                                                                                    
    return first+second+third
#Formula for cosθ and cosθz with β varying

In [25]:
cosθ=value(15)
df['cosθ']=cosθ
#Incidence angles and saving to the dataframe

In [26]:
cosθz=value(0)
df['cosθz']=cosθz
#Solar zenith angles and saving to the dataframe

In [27]:
rb=cosθ/cosθz
df['rb']=rb
#Finding the tilt factor for beam radiation and saving to the dataframe

In [28]:
max(rb)
#Weird value noted meansd more data cleaning needed, will do after finding It

45025.013979347415

In [29]:
rd=(1+np.cos(np.deg2rad(β)))/2
#Finding the tilt factor for diffused radiation

In [30]:
rr=0.2*(1-np.cos(np.deg2rad(β)))/2
#Finding the tilt factor for reflected radiation

In [31]:
It = np.multiply(Ib,rb) + Id*rd + Ig*rr
df['It']=It
#Finding the tilted radiation and saving to the dataframe

In [32]:
df['Ibn']=df['BHI']/df['cosθz']
#Finding the beam normal radiation

In [33]:
df

,Hour,GHI,DHI,Temperature,Wind Speed,BHI,day,I_sc',δ,daywise_hours,Error in min,LAT_minutes,LAT,ω,cosθ,cosθz,rb,It,Ibn
0,1.0,0.0,0.0,11.0,0.2,0.0,1,1412.104316,-23.011637,1,-2.904169,19.295831,00:19,-175.176042,-0.976979,-0.995142,0.981749,0.0,-0.0
1,2.0,0.0,0.0,10.0,0.2,0.0,1,1412.104316,-23.011637,2,-2.904169,79.295831,01:19,-160.176042,-0.926739,-0.949279,0.976256,0.0,-0.0
2,3.0,0.0,0.0,10.0,0.2,0.0,1,1412.104316,-23.011637,3,-2.904169,139.295831,02:19,-145.176042,-0.818692,-0.850645,0.962437,0.0,-0.0
3,4.0,0.0,0.0,9.0,0.2,0.0,1,1412.104316,-23.011637,4,-2.904169,199.295831,03:19,-130.176042,-0.660202,-0.705963,0.935179,0.0,-0.0
4,5.0,0.0,0.0,9.0,0.2,0.0,1,1412.104316,-23.011637,5,-2.904169,259.295831,04:19,-115.176042,-0.462069,-0.525092,0.879977,0.0,-0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,8756.0,0.0,0.0,15.0,0.3,0.0,365,1412.111000,-23.085911,20,-2.453134,1159.746866,19:19,109.936716,-0.386020,-0.455984,0.846565,0.0,-0.0
8756,8757.0,0.0,0.0,14.0,0.3,0.0,365,1412.111000,-23.085911,21,-2.453134,1219.746866,20:19,124.936716,-0.594815,-0.646588,0.919929,0.0,-0.0
8757,8758.0,0.0,0.0,13.0,0.3,0.0,365,1412.111000,-23.085911,22,-2.453134,1279.746866,21:19,139.936716,-0.768439,-0.805085,0.954481,0.0,-0.0
8758,8759.0,0.0,0.0,12.0,0.3,0.0,365,1412.111000,-23.085911,23,-2.453134,1339.746866,22:19,154.936716,-0.895061,-0.920675,0.972178,0.0,-0.0


## More Data Cleaning

In [34]:
df.sort_values(by='It', ascending=False)

,Hour,GHI,DHI,Temperature,Wind Speed,BHI,day,I_sc',δ,daywise_hours,Error in min,LAT_minutes,LAT,ω,cosθ,cosθz,rb,It,Ibn
305,306.0,51.0,44.0,17.0,0.3,7.0,13,1410.986136,-21.596777,18,-7.901680,1034.298320,17:14,78.574580,0.106528,0.000002,45025.013979,315218.522000,2.958610e+06
2004,2005.0,949.0,250.0,37.0,0.2,699.0,84,1372.615384,1.210481,13,-6.610441,735.589559,12:15,3.897390,0.981392,0.901471,1.088656,1009.945167,7.753995e+02
1908,1909.0,937.0,249.0,39.0,0.1,688.0,80,1375.681683,-0.403653,13,-7.861940,734.338060,12:14,3.584515,0.976278,0.889360,1.097731,1003.189568,7.735899e+02
2700,2701.0,983.0,294.0,44.0,0.1,689.0,113,1350.501891,12.274096,13,1.650186,743.850186,12:23,5.962547,0.994749,0.964248,1.031631,1003.134413,7.145462e+02
2724,2725.0,984.0,294.0,45.0,0.0,690.0,114,1349.781618,12.616221,13,1.851597,744.051597,12:24,6.012899,0.994578,0.965633,1.029975,1003.026740,7.145571e+02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3787,3788.0,0.0,0.0,37.0,0.5,0.0,158,1325.841862,22.747999,20,1.576165,1163.776165,19:23,110.944041,-0.245295,-0.121763,2.014520,0.000000,-0.000000e+00
3773,3774.0,0.0,0.0,31.0,0.4,0.0,158,1325.841862,22.747999,6,1.576165,323.776165,05:23,-99.055959,-0.064557,0.043228,-1.493411,0.000000,0.000000e+00
233,234.0,53.0,43.0,22.0,0.1,10.0,10,1411.444264,-22.039625,18,-6.741642,1035.458358,17:15,78.864589,0.100022,-0.007832,-12.770624,-85.258238,-1.276775e+03
257,258.0,50.0,43.0,23.0,0.2,7.0,11,1411.304668,-21.898483,18,-7.136171,1035.063829,17:15,78.765957,0.102191,-0.005249,-19.466760,-93.829540,-1.333463e+03


Clearly It max value is coming out to be 315218 W/m2 which is way out of bound, so we replace it with zero. Negative value sof It are also replaced with zero

In [35]:
It[It<0]=0
It[It>1100]=0
df['It']=It

I_t is cleaned now<br>
Let's clean I_bn

In [36]:
Ibn=np.asarray(df['Ibn'])
Ibn[Ibn<0]=0
df['Ibn']=Ibn

In [37]:
df.sort_values(by='Ibn', ascending=False)

,Hour,GHI,DHI,Temperature,Wind Speed,BHI,day,I_sc',δ,daywise_hours,Error in min,LAT_minutes,LAT,ω,cosθ,cosθz,rb,It,Ibn
305,306.0,51.0,44.0,17.0,0.3,7.0,13,1410.986136,-21.596777,18,-7.901680,1034.298320,17:14,78.574580,0.106528,0.000002,45025.013979,0.000000,2.958610e+06
7001,7002.0,52.0,48.0,30.0,0.1,4.0,292,1380.940066,-11.048690,18,15.192068,1057.392068,17:37,84.348017,0.056211,0.000682,82.452603,377.169820,5.867350e+03
329,330.0,55.0,47.0,14.0,0.1,8.0,14,1410.807294,-21.436301,18,-8.271976,1033.928024,17:13,78.482006,0.108695,0.002668,40.733039,372.250976,2.997982e+03
353,354.0,58.0,50.0,14.0,0.2,8.0,15,1410.615472,-21.269474,18,-8.633528,1033.566472,17:13,78.391618,0.110857,0.005359,20.684964,214.825485,1.492726e+03
377,378.0,62.0,52.0,15.0,0.3,10.0,16,1410.410725,-21.096344,18,-8.986017,1033.213983,17:13,78.303496,0.113015,0.008073,13.998388,191.309207,1.238630e+03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3721,3722.0,0.0,0.0,33.0,0.5,0.0,156,1326.501898,22.538493,2,1.932589,84.132589,01:24,-158.966853,-0.767562,-0.599420,1.280507,0.000000,-0.000000e+00
3720,3721.0,0.0,0.0,34.0,0.6,0.0,156,1326.501898,22.538493,1,1.932589,24.132589,00:24,-173.966853,-0.822835,-0.649878,1.266138,0.000000,-0.000000e+00
3719,3720.0,0.0,0.0,35.0,0.6,0.0,155,1326.849966,22.423707,24,2.102571,1404.302571,23:24,171.075643,-0.818006,-0.645959,1.266344,0.000000,-0.000000e+00
3718,3719.0,0.0,0.0,36.0,0.5,0.0,155,1326.849966,22.423707,23,2.102571,1344.302571,22:24,156.075643,-0.751164,-0.584941,1.284172,0.000000,-0.000000e+00


In [38]:
df
#Final dataframe

,Hour,GHI,DHI,Temperature,Wind Speed,BHI,day,I_sc',δ,daywise_hours,Error in min,LAT_minutes,LAT,ω,cosθ,cosθz,rb,It,Ibn
0,1.0,0.0,0.0,11.0,0.2,0.0,1,1412.104316,-23.011637,1,-2.904169,19.295831,00:19,-175.176042,-0.976979,-0.995142,0.981749,0.0,-0.0
1,2.0,0.0,0.0,10.0,0.2,0.0,1,1412.104316,-23.011637,2,-2.904169,79.295831,01:19,-160.176042,-0.926739,-0.949279,0.976256,0.0,-0.0
2,3.0,0.0,0.0,10.0,0.2,0.0,1,1412.104316,-23.011637,3,-2.904169,139.295831,02:19,-145.176042,-0.818692,-0.850645,0.962437,0.0,-0.0
3,4.0,0.0,0.0,9.0,0.2,0.0,1,1412.104316,-23.011637,4,-2.904169,199.295831,03:19,-130.176042,-0.660202,-0.705963,0.935179,0.0,-0.0
4,5.0,0.0,0.0,9.0,0.2,0.0,1,1412.104316,-23.011637,5,-2.904169,259.295831,04:19,-115.176042,-0.462069,-0.525092,0.879977,0.0,-0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,8756.0,0.0,0.0,15.0,0.3,0.0,365,1412.111000,-23.085911,20,-2.453134,1159.746866,19:19,109.936716,-0.386020,-0.455984,0.846565,0.0,-0.0
8756,8757.0,0.0,0.0,14.0,0.3,0.0,365,1412.111000,-23.085911,21,-2.453134,1219.746866,20:19,124.936716,-0.594815,-0.646588,0.919929,0.0,-0.0
8757,8758.0,0.0,0.0,13.0,0.3,0.0,365,1412.111000,-23.085911,22,-2.453134,1279.746866,21:19,139.936716,-0.768439,-0.805085,0.954481,0.0,-0.0
8758,8759.0,0.0,0.0,12.0,0.3,0.0,365,1412.111000,-23.085911,23,-2.453134,1339.746866,22:19,154.936716,-0.895061,-0.920675,0.972178,0.0,-0.0


In [39]:
df.to_excel('data.xlsx')

# Modelling

In [40]:
from CoolProp.CoolProp import PhaseSI, PropsSI, get_global_param_string
import CoolProp.CoolProp as cp

In [41]:
h1= cp.PropsSI('H', 'T',10+273,'P', 400000,'Water')
h2= cp.PropsSI('H', 'T',90+273,'P', 100000,'Water')

In [42]:
T1=10+273
T2=90+273
T3=T2
T4=T2
T5=T1
T6=5+273
T7=85+273

Assuming ideal heat exchanger <br>
Using m2Cpw(T7-T6)=m3Cpm(T4-T5) we find m2

In [43]:
from sympy import symbols, solve

In [44]:
m = symbols('x')

In [45]:
m3=20*1032

In [46]:
expr=m*4.2*(T7-T6)-m3*3.93*(T4-T5)

In [47]:
print("The value of the volume flow rate of water in the heat exchanger in m3/hr is", solve(expr)[0]/1000)

The value of the volume flow rate of water in the heat exchanger in m3/hr is 19.3131428571429


In [48]:
print("Useful energy needed for pasteurization is", m3*3.93*(T4-T5), "in kJ/hr")

Useful energy needed for pasteurization is 6489216.0 in kJ/hr


We'll model for June 21st

In [49]:
n=173

In [50]:
dff=df.iloc[(n-1)*24:n*24]
#Selecting the values for June 21st

In [51]:
dff

,Hour,GHI,DHI,Temperature,Wind Speed,BHI,day,I_sc',δ,daywise_hours,Error in min,LAT_minutes,LAT,ω,cosθ,cosθz,rb,It,Ibn
4128,4129.0,0.0,0.0,32.0,0.6,0.0,173,1322.490874,23.448046,1,-1.544107,20.655893,00:20,-174.836027,-0.815190,-0.639038,1.275652,0.000000,-0.000000
4129,4130.0,0.0,0.0,31.0,0.6,0.0,173,1322.490874,23.448046,2,-1.544107,80.655893,01:20,-159.836027,-0.763754,-0.592083,1.289943,0.000000,-0.000000
4130,4131.0,0.0,0.0,30.0,0.6,0.0,173,1322.490874,23.448046,3,-1.544107,140.655893,02:20,-144.836027,-0.654825,-0.492644,1.329204,0.000000,-0.000000
4131,4132.0,0.0,0.0,29.0,0.6,0.0,173,1322.490874,23.448046,4,-1.544107,200.655893,03:20,-129.836027,-0.495825,-0.347497,1.426846,0.000000,-0.000000
4132,4133.0,0.0,0.0,29.0,0.6,0.0,173,1322.490874,23.448046,5,-1.544107,260.655893,04:20,-114.836027,-0.297591,-0.166534,1.786968,0.000000,-0.000000
4133,4134.0,0.0,0.0,29.0,0.5,0.0,173,1322.490874,23.448046,6,-1.544107,320.655893,05:20,-99.836027,-0.073631,0.037913,-1.942096,0.000000,0.000000
4134,4135.0,54.0,51.0,30.0,0.5,3.0,173,1322.490874,23.448046,7,-1.544107,380.655893,06:20,-84.836027,0.160791,0.251912,0.638282,52.229955,11.908919
4135,4136.0,245.0,152.0,32.0,0.5,93.0,173,1322.490874,23.448046,8,-1.544107,440.655893,07:20,-69.836027,0.389701,0.460878,0.845561,228.882355,201.788618
4136,4137.0,453.0,192.0,34.0,0.5,261.0,173,1322.490874,23.448046,9,-1.544107,500.655893,08:20,-54.836027,0.597498,0.650572,0.918420,429.980119,401.185703
4137,4138.0,658.0,238.0,37.0,0.5,420.0,173,1322.490874,23.448046,10,-1.544107,560.655893,09:20,-39.836027,0.770022,0.808064,0.952921,636.414220,519.760559


At LAT close to 12 noon, 12:20PM, Ibn is maximum which is 687.3W/m2

In [52]:
I_bn=687.3
A=19.313*1000*(h2-h1)/(I_bn*0.65*3600)
print(A)
# Returns area needed for the solar field fot hat time instant

4018.6409398989103


In [53]:
mw1=19.313*Ibn/I_bn
df['mw1']=mw1
#Finding the mass flow rae for all the other hours based on these values

In [54]:
df

,Hour,GHI,DHI,Temperature,Wind Speed,BHI,day,I_sc',δ,daywise_hours,Error in min,LAT_minutes,LAT,ω,cosθ,cosθz,rb,It,Ibn,mw1
0,1.0,0.0,0.0,11.0,0.2,0.0,1,1412.104316,-23.011637,1,-2.904169,19.295831,00:19,-175.176042,-0.976979,-0.995142,0.981749,0.0,-0.0,-0.0
1,2.0,0.0,0.0,10.0,0.2,0.0,1,1412.104316,-23.011637,2,-2.904169,79.295831,01:19,-160.176042,-0.926739,-0.949279,0.976256,0.0,-0.0,-0.0
2,3.0,0.0,0.0,10.0,0.2,0.0,1,1412.104316,-23.011637,3,-2.904169,139.295831,02:19,-145.176042,-0.818692,-0.850645,0.962437,0.0,-0.0,-0.0
3,4.0,0.0,0.0,9.0,0.2,0.0,1,1412.104316,-23.011637,4,-2.904169,199.295831,03:19,-130.176042,-0.660202,-0.705963,0.935179,0.0,-0.0,-0.0
4,5.0,0.0,0.0,9.0,0.2,0.0,1,1412.104316,-23.011637,5,-2.904169,259.295831,04:19,-115.176042,-0.462069,-0.525092,0.879977,0.0,-0.0,-0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,8756.0,0.0,0.0,15.0,0.3,0.0,365,1412.111000,-23.085911,20,-2.453134,1159.746866,19:19,109.936716,-0.386020,-0.455984,0.846565,0.0,-0.0,-0.0
8756,8757.0,0.0,0.0,14.0,0.3,0.0,365,1412.111000,-23.085911,21,-2.453134,1219.746866,20:19,124.936716,-0.594815,-0.646588,0.919929,0.0,-0.0,-0.0
8757,8758.0,0.0,0.0,13.0,0.3,0.0,365,1412.111000,-23.085911,22,-2.453134,1279.746866,21:19,139.936716,-0.768439,-0.805085,0.954481,0.0,-0.0,-0.0
8758,8759.0,0.0,0.0,12.0,0.3,0.0,365,1412.111000,-23.085911,23,-2.453134,1339.746866,22:19,154.936716,-0.895061,-0.920675,0.972178,0.0,-0.0,-0.0


In [55]:
SM=1.8
print("The area of solar field needed is", SM*A)
#Final area requirement for the whole solar field

The area of solar field needed is 7233.553691818039
